In [10]:
import pandas as pd

deposit_data = pd.read_excel("Deposit_data.xlsx",skiprows=3)
withdrawal_data = pd.read_excel("Withdrawal_data.xlsx",skiprows=3)
game_data = pd.read_excel("user_gameplay_data.xlsx",skiprows=3)

print("Deposit Data:\n", deposit_data.head())
print("Withdrawal Data:\n", withdrawal_data.head())
print("Game Data:\n", game_data.head())


Deposit Data:
    User Id             Datetime  Amount
0      357  2022-01-10 00:03:00    2000
1      776  2022-01-10 00:03:00    2500
2      492  2022-01-10 00:06:00    5000
3      803  2022-01-10 00:07:00    5000
4      875  2022-01-10 00:09:00    1500
Withdrawal Data:
    User Id             Datetime  Amount
0      190  2022-01-10 00:03:00    5872
1      159  2022-01-10 00:16:00    9540
2      164  2022-01-10 00:24:00     815
3      946  2022-01-10 00:29:00   23000
4      763  2022-01-10 00:40:00    9473
Game Data:
    User ID  Games Played             Datetime
0      851             1  2022-01-10 00:00:00
1      717             1  2022-01-10 00:00:00
2      456             1  2022-01-10 00:00:00
3      424             1  2022-01-10 00:00:00
4      845             1  2022-01-10 00:00:00


In [11]:
deposit_data['Datetime'] = pd.to_datetime(deposit_data['Datetime'])
withdrawal_data['Datetime'] = pd.to_datetime(withdrawal_data['Datetime'])
game_data['Datetime'] = pd.to_datetime(game_data['Datetime'])

In [12]:
average_deposit = deposit_data['Amount'].mean()
print(f"The average deposit amount is: {average_deposit:.2f}")

The average deposit amount is: 5492.19


In [13]:
deposit_data['YearMonth'] = deposit_data['Datetime'].dt.to_period('M')

average_deposit_per_user = deposit_data.groupby(['User Id', 'YearMonth'])['Amount'].mean().reset_index()

print(average_deposit_per_user)

      User Id YearMonth        Amount
0           1   2022-10   5000.000000
1           2   2022-01  40000.000000
2           2   2022-02   5000.000000
3           2   2022-06  40000.000000
4           2   2022-08  16666.666667
...       ...       ...           ...
4343      996   2022-12   5000.000000
4344      998   2022-04   1000.000000
4345      998   2022-10   1600.000000
4346      999   2022-01   2000.000000
4347      999   2022-10   3500.000000

[4348 rows x 3 columns]


In [17]:
print(game_data.columns)

Index(['User ID', 'Games Played', 'Datetime'], dtype='object')


In [22]:
average_games_per_user = game_data.groupby('User ID')['Games Played'].mean().reset_index()
average_games_per_user.columns = ['User ID', 'Average Games Played']
print(average_games_per_user)

     User ID  Average Games Played
0          0                   1.0
1          1                   1.0
2          2                   1.0
3          3                   1.0
4          4                   1.0
..       ...                   ...
995      995                   1.0
996      996                   1.0
997      997                   1.0
998      998                   1.0
999      999                   1.0

[1000 rows x 2 columns]


In [27]:

print("Deposit Data Columns:", deposit_data.columns)
print("Withdrawal Data Columns:", withdrawal_data.columns)
print("Game Data Columns:", game_data.columns)



Deposit Data Columns: Index(['User Id', 'Datetime', 'Amount', 'YearMonth'], dtype='object')
Withdrawal Data Columns: Index(['User Id', 'Datetime', 'Amount'], dtype='object')
Game Data Columns: Index(['User ID', 'Games Played', 'Datetime'], dtype='object')


In [28]:
slots = {
    'S1': {'start': '00:00:00', 'end': '12:00:00'},
    'S2': {'start': '12:00:00', 'end': '23:59:59'}
}

dates_slots = [
    {'date': '2022-10-02', 'slot': 'S1'},
    {'date': '2022-10-16', 'slot': 'S2'},
    {'date': '2022-10-18', 'slot': 'S1'},
    {'date': '2022-10-26', 'slot': 'S2'}
]

def filter_by_slot(data, date, slot):
    start_time = pd.to_datetime(f"{date} {slots[slot]['start']}")
    end_time = pd.to_datetime(f"{date} {slots[slot]['end']}")
    return data[(data['Datetime'] >= start_time) & (data['Datetime'] <= end_time)]


results = []


for ds in dates_slots:
    date = ds['date']
    slot = ds['slot']

    deposit_filtered = filter_by_slot(deposit_data, date, slot)
    withdrawal_filtered = filter_by_slot(withdrawal_data, date, slot)
    games_filtered = filter_by_slot(game_data, date, slot)

    users = set(deposit_filtered['User Id']).union(withdrawal_filtered['User Id']).union(games_filtered['User ID'])
    for user_id in users:
        deposit_amount = deposit_filtered[deposit_filtered['User Id'] == user_id]['Amount'].sum()
        withdrawal_amount = withdrawal_filtered[withdrawal_filtered['User Id'] == user_id]['Amount'].sum()
        num_deposits = deposit_filtered[deposit_filtered['User Id'] == user_id].shape[0]
        num_withdrawals = withdrawal_filtered[withdrawal_filtered['User Id'] == user_id].shape[0]
        games_played = games_filtered[games_filtered['User ID'] == user_id]['Games Played'].sum()
       
        deposit_points = 0.01 * deposit_amount
        withdrawal_points = 0.005 * withdrawal_amount
        frequency_points = 0.001 * max(num_deposits - num_withdrawals, 0)
        games_points = 0.2 * games_played
       
        total_points = deposit_points + withdrawal_points + frequency_points + games_points

        results.append({
            'User ID': user_id,
            'Date': date,
            'Slot': slot,
            'Deposit Points': deposit_points,
            'Withdrawal Points': withdrawal_points,
            'Frequency Points': frequency_points,
            'Games Points': games_points,
            'Total Points': total_points
        })

loyalty_points_df = pd.DataFrame(results)
loyalty_points_df.sort_values(by=['Date', 'Slot', 'Total Points'], ascending=[True, True, False], inplace=True)
loyalty_points_df.reset_index(drop=True, inplace=True)

loyalty_points_df.head(10)


,User ID,Date,Slot,Deposit Points,Withdrawal Points,Frequency Points,Games Points,Total Points
0,634,2022-10-16,S2,0.00,1491.555,0.000,0.0,1491.555
1,212,2022-10-16,S2,999.99,0.000,0.001,0.0,999.991
2,99,2022-10-16,S2,980.00,0.000,0.002,0.0,980.002
3,28,2022-10-16,S2,900.00,0.000,0.004,0.0,900.004
4,566,2022-10-16,S2,880.00,0.000,0.003,0.2,880.203
5,714,2022-10-16,S2,610.00,0.000,0.001,0.0,610.001
6,786,2022-10-16,S2,0.00,600.000,0.000,0.0,600.000
7,455,2022-10-16,S2,400.00,0.000,0.002,2.0,402.002
8,175,2022-10-16,S2,0.00,400.000,0.000,0.0,400.000
9,82,2022-10-16,S2,0.00,375.550,0.000,0.0,375.550


In [29]:

october_start = pd.to_datetime("2022-10-01")
october_end = pd.to_datetime("2022-10-31")


deposit_october = deposit_data[(deposit_data['Datetime'] >= october_start) & (deposit_data['Datetime'] <= october_end)]
withdrawal_october = withdrawal_data[(withdrawal_data['Datetime'] >= october_start) & (withdrawal_data['Datetime'] <= october_end)]
games_october = game_data[(game_data['Datetime'] >= october_start) & (game_data['Datetime'] <= october_end)]


user_points = {}

all_users = set(deposit_october['User Id']).union(withdrawal_october['User Id']).union(games_october['User ID'])

for user_id in all_users:
    deposit_amount = deposit_october[deposit_october['User Id'] == user_id]['Amount'].sum()
    withdrawal_amount = withdrawal_october[withdrawal_october['User Id'] == user_id]['Amount'].sum()
    num_deposits = deposit_october[deposit_october['User Id'] == user_id].shape[0]
    num_withdrawals = withdrawal_october[withdrawal_october['User Id'] == user_id].shape[0]
    games_played = games_october[games_october['User ID'] == user_id]['Games Played'].sum()
    
    # Calculate individual components of loyalty points
    deposit_points = 0.01 * deposit_amount
    withdrawal_points = 0.005 * withdrawal_amount
    frequency_points = 0.001 * max(num_deposits - num_withdrawals, 0)
    games_points = 0.2 * games_played
    
    # Total loyalty points
    total_points = deposit_points + withdrawal_points + frequency_points + games_points

    # Store results in dictionary
    user_points[user_id] = {
        'Total Points': total_points,
        'Games Played': games_played
    }

loyalty_points_october_df = pd.DataFrame.from_dict(user_points, orient='index').reset_index()
loyalty_points_october_df.rename(columns={'index': 'User ID'}, inplace=True)


loyalty_points_october_df.sort_values(by=['Total Points', 'Games Played'], ascending=[False, False], inplace=True)
loyalty_points_october_df['Rank'] = range(1, len(loyalty_points_october_df) + 1)

loyalty_points_october_df.head(10)


,User ID,Total Points,Games Played,Rank
632,634,58121.295,21,1
712,714,14180.823,4,2
210,212,13447.226,0,3
670,672,12338.422,7,4
97,99,11989.946,4,5
564,566,11620.181,100,6
30,30,10552.560,9,7
367,369,10344.772,21,8
567,569,9942.418,23,9
738,740,9821.880,2,10


In [30]:

top_50_players = loyalty_points_october_df.head(50)

top_50_players['Weighted Score'] = top_50_players['Total Points'] + (0.1 * top_50_players['Games Played'])

total_weighted_score = top_50_players['Weighted Score'].sum()

top_50_players['Bonus Share (%)'] = (top_50_players['Weighted Score'] / total_weighted_score) * 100

total_bonus_pool = 50000
top_50_players['Bonus Amount'] = (top_50_players['Bonus Share (%)'] / 100) * total_bonus_pool

top_50_players.sort_values(by='Rank', inplace=True)
top_50_players[['User ID', 'Rank', 'Total Points', 'Games Played', 'Weighted Score', 'Bonus Amount']]


C:\Users\patna\AppData\Local\Temp\ipykernel_14808\1527077487.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50_players['Weighted Score'] = top_50_players['Total Points'] + (0.1 * top_50_players['Games Played'])
C:\Users\patna\AppData\Local\Temp\ipykernel_14808\1527077487.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50_players['Bonus Share (%)'] = (top_50_players['Weighted Score'] / total_weighted_score) * 100
C:\Users\patna\AppData\Local\Temp\ipykernel_14808\1527077487.py:15: SettingWit

,User ID,Rank,Total Points,Games Played,Weighted Score,Bonus Amount
632,634,1,58121.295,21,58123.395,7199.136312
712,714,2,14180.823,4,14181.223,1756.479597
210,212,3,13447.226,0,13447.226,1665.567075
670,672,4,12338.422,7,12339.122,1528.317836
97,99,5,11989.946,4,11990.346,1485.118605
564,566,6,11620.181,100,11630.181,1440.508737
30,30,7,10552.560,9,10553.460,1307.146582
367,369,8,10344.772,21,10346.872,1281.558690
567,569,9,9942.418,23,9944.718,1231.748085
738,740,10,9821.880,2,9822.080,1216.558200
